# Part 1 - Importing necessary packages

If installation was done correctly, there should be no errors here.

In [ ]:
%matplotlib inline

# Numerical library
import numpy as np

# Data manipulation
import pandas as pd
from patsy import dmatrix

# Ploting
import matplotlib
import matplotlib.pyplot as plt

# Survival analysis
import lifelines

# Part 2 - A look at the clinical data
The data is originally avaliable at http://www.cbioportal.org/study?id=brca_tcga_pub, but it can also be found together with this notebook at https://github.com/gjeuken/survival-workshop.

We will use here clinical data on the patients as well as gene expression data. Firstly, lets load the clinical data and see what it looks like. Make a note of all the variables (columns) avaliable in the data.

In [ ]:
clinical = pd.read_csv('data/brca_tcga_pub_clinical_data.tsv', sep='\t')

with pd.option_context('display.max_columns', None):
    display(clinical)

To make the later survival analysis easier, we create two new columns, one for the survival in days, and one for the event (death) beign present.

In [ ]:
clinical['Time'] = clinical['Overall Survival (Months)']*30
clinical['Event'] = (clinical['Overall Survival Status'] == 'DECEASED')*1

Lets plot how the survival data looks like

In [ ]:
matplotlib.rcParams['figure.figsize'] = [15, 50]

data_sorted = clinical[['Time','Event']].sort_values(by = 'Time').dropna().reset_index(drop=True)
status_slice = data_sorted['Event'] == 1

plt.barh(data_sorted.loc[~status_slice].index, data_sorted.loc[~status_slice,'Time'], height = 1, color = 'b')
plt.barh(data_sorted.loc[status_slice].index, data_sorted.loc[status_slice,'Time'], height = 1, color = 'r')
plt.legend(['Alive', 'Dead'])
plt.ylabel('Patients')
plt.xlabel('Days alive')

---
### Exercises
1.1 Describe the plot and what inferences you are able to make from it

---
# Part 3 - Survival analysis using the clinical data



Ok, now that we've seen the data, lets play around with it.

How does the survival curve looks like in general? We can use the survival package __lifelines__ to figure this out, and generate a *Kaplan Meier plot*. 

Note that in the plot the shaded area represents a 95% confidence interval.

In [ ]:
kmf = lifelines.KaplanMeierFitter()

clinical = clinical.dropna()

kmf.fit(clinical['Time'], clinical['Event'])

matplotlib.rcParams['figure.figsize'] = [15, 10]
kmf.plot()
plt.xlabel('Days alive')
plt.ylabel('Fraction alive')

---
### Exercises

2.1 Compare the Kaplan Meier plot with the first one, what additional insights are avaliable on this latter plot?

2.2 Why is there a confidence interval? Aren't we just displaying the data?

---

Now we can start to play around with clinical variables that might influence in the survival curve.

You should play around with the groupings and see if you can find some usefull insight

In [ ]:
# Define groups here
group_1 = clinical.loc[clinical['Metastasis-Coded'] == 'Positive']
group_2 = clinical.loc[clinical['Metastasis-Coded'] == 'Negative']

kmf.fit(group_1['Time'], group_1['Event'], label='Positive')
ax = kmf.plot(ci_show=False)

kmf.fit(group_2['Time'], group_2['Event'], label='Negative')
kmf.plot(ax=ax, ci_show=False)

plt.title('Metastasis')

---
### Exercises
3.1 Make at least 3 plots using different separation criteria, and lable them accordingly.

3.2 Make a plot that includes the survival curve for 3 different age groups.

3.3 (Advanced) The [**logrank_test** function](http://lifelines.readthedocs.io/en/latest/Examples.html?compare-using-a-hypothesis-test#compare-using-a-hypothesis-test) of the **lifelines** package performs a statistical test on the two groups to see if their event (death) generation process is the same. Use this function to obtain a significance statistic for the separations above.

---

# Part 4 - Cox Proportional Hazards model

The next step is to use the Cox Proportional Hazards (CPH) in the data. 

First, let's check the main assumption of the model, the proportionality of hazards, by ploting the hazard functions on a log-log scale and seeing wether they "look" parallel or not.

In [ ]:
# Define groups here
group_1 = clinical.loc[clinical['Metastasis-Coded'] == 'Positive']
group_2 = clinical.loc[clinical['Metastasis-Coded'] == 'Negative']

kmf.fit(group_1['Time'], group_1['Event'], label='Positive')
ax = kmf.plot_loglogs()

kmf.fit(group_2['Time'], group_2['Event'], label='Negative')
kmf.plot_loglogs(ax=ax)

---
### Exercises

4.1 In your oppinion, does the proportional hazards assumption hold for the Metastasis case? What about other variables you used on the previous exercise? Use the plots to support your claims.

---

Now that we checked the proportional hazards assumption, we can perform the CPH regression on the categorical data where this assumption holds.

Note that here we will deal with the [patsy package](https://patsy.readthedocs.io/en/latest/) to format our categorical data in a way that is suited for the regression. If you are not familiar with this type of representation, please take a moment to familiriarize yourself in the [relevant help page](https://patsy.readthedocs.io/en/latest/categorical-coding.html).




In [ ]:
cph = lifelines.CoxPHFitter()

formula = 'Time + Event + C(Q("Metastasis"), levels = l) + Q("Diagnosis Age")'

l = ['M0', 'M1']


cox_input = dmatrix(formula, data = clinical, return_type = 'dataframe')
cox_input = cox_input.drop(['Intercept'], axis = 1)

cph.fit(cox_input, duration_col='Time', event_col='Event')

cph.plot()
cph.summary

---
### Exercises
5.1 Perform the CPH regression using a few variables where the proportional hazard assumption hold.

5.2 Try adding more than one variable in the same regression. What happens then? Explain the results.

5.3 If there are variables where the proportional hazard assumption does not hold, but hold predictive power, you might still use it to [stratify your data](http://lifelines.readthedocs.io/en/latest/Survival%20Regression.html#stratification). How do the results change then?

5.4 Can you find a way to plot the survival curves predicted by the regression for each level of a categorical variable? (Remember the CPH assumpions)

---

# Part 5 - Incorporating gene expression data

It's time to look at the gene expression data!

First we load the expression data for the same samples. This may take some time, be patient.

We then take a look on how the data looks like. Note that the gene expression has been median normalized, so we get negative gene expression here.

In [ ]:
expression_raw  = pd.read_csv('data/data_expression_median.txt', sep='\t')
expression = expression_raw.set_index('Hugo_Symbol').iloc[:,1:].T
expression

Now we merge the clinical and expression data in one big datastructure, and display the result.

In [ ]:
data = clinical.merge(expression, how='inner', left_on='Sample ID', right_index=True)
data

Now we can incorporate the gene expression data into our Cox regression

In [ ]:
formula = "Q('Diagnosis Age') -1"

X = dmatrix(formula, data)
X